## Requirements

In [1]:
!pip install -U openai-whisper
!pip install openai
!pip install langchain
!pip install git+https://github.com/suno-ai/bark.git
!pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798399 sha256=795c9ebc6a3d1708d1267bd9b92f9e2c32fb1ae00e2476cc3a8b441a6972f560
  Stored in directory: /root/.cache/pip/wheels/5d/37/b1/9aea93201fe91e3561719120da92cc23e77b7ef6f3d0d9491a
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-mzh_k5t

## Import Libraries

In [10]:
import whisper
import openai
import langchain
from langchain.chains import SequentialChain,LLMChain,SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import os
from bark import SAMPLE_RATE, generate_audio
from IPython.display import Audio
from langchain.memory import ConversationBufferWindowMemory

##OpenAI STT Model: Whisper

In [33]:
STT_model=whisper.load_model('medium.en')

def speach_to_text(voice):
  result=STT_model.transcribe(voice)
  transcript=result['text']

  return transcript

100%|█████████████████████████████████████| 1.42G/1.42G [00:56<00:00, 27.1MiB/s]


Enter your OpenAI API key below:

In [5]:
os.environ['OPENAI_API_KEY']="xxxxxxxxxxxxxxxxxxxxxxx" #OpenAI API KEY

#Grammer


In [48]:

grammer_prompt=PromptTemplate(template="""
You are an advanced English teacher. Your task is to say MAX 10 grammerical mistakes and correct version of them in the following text in numbered form:\n
{original_text}
""",
input_variables=['original_text'])

grammer_llm=OpenAI(model='gpt-3.5-turbo-instruct',temperature=0.2,max_tokens=350)

grammer_chain=LLMChain(llm=grammer_llm,prompt=grammer_prompt,output_key='grammer')

#Speaking

In [22]:
#memory = ConversationBufferWindowMemory(k=3)

speaking_prompt=PromptTemplate(template="You are an advanced Speaking English teacher.your student send a text below and he wants to strat a converstion with you.Talk about subject below and finish your sentence with a interesting question :\n {text}",
               input_variables=['text'])

speaking_llm=OpenAI(model='gpt-3.5-turbo-instruct',temperature=0.7,max_tokens=500)

speaking_chain=LLMChain(llm=speaking_llm,prompt=speaking_prompt,output_key='speaking')

#Chain

In [49]:
def tts_model(voice):
  text_output=speach_to_text(voice)
  chain=SequentialChain(chains=[grammer_chain,speaking_chain],
                        input_variables=['text','original_text'],
                        output_variables=['grammer','speaking'])


  result=chain({'text':text_output,'original_text':text_output})

  grammer_output=result['grammer'].strip().split("\n")

  speaking_output=result['speaking'].strip()

  audio_array = generate_audio(speaking_output)

  return grammer_output,Audio(audio_array, rate=SAMPLE_RATE)


## Results

In [50]:
g,s=tts_model('/content/Recording (4) - Copy.mp3')

100%|██████████| 26/26 [00:23<00:00,  1.12it/s]


In [51]:
g

['1. "One more week" should be "After one more week".',
 '2. "The puppies began to walk" should be "The puppies started walking".',
 '3. "Everyone left" should be "Everyone was amazed".',
 '4. "When they watched the little dogs" should be "As they watched the little dogs".',
 '5. "Because they couldn\'t not walk well" should be "Because they couldn\'t walk well".',
 '6. "Two or three weeks later" should be "After two or three weeks".',
 '7. "When the puppies were bigger" should be "As the puppies grew bigger".',
 '8. "Could walk better" should be "Could walk more easily".',
 '9. "They play a lot" should be "They played a lot".',
 '10. "The children love to play with them" should be "The children loved playing with them".']

In [ ]:
s